In [1]:
import logging
#logging.getLogger("imperative_model").setLevel(logging.DEBUG)
#logging.basicConfig(level=logging.DEBUG)

In [12]:
%load_ext autoreload
%autoreload 2
%run load_model.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
model.processes

[Process(id='CCGT', produces=['Electricity'], consumes=['NaturalGas'], has_stock=False),
 Process(id='ElectricCarUse', produces=[], consumes=['Electricity'], has_stock=False),
 Process(id='HydrogenElectrolysis', produces=['Hydrogen'], consumes=['Electricity'], has_stock=False),
 Process(id='SteelProductionEAF', produces=['Steel'], consumes=['Electricity'], has_stock=False),
 Process(id='SteelProductionH2DRI', produces=['Steel'], consumes=['Hydrogen'], has_stock=False),
 Process(id='WindTurbine', produces=['Electricity'], consumes=[], has_stock=False)]

In [14]:
model.objects

[Object(id='Electricity', has_market=True),
 Object(id='Hydrogen', has_market=True),
 Object(id='NaturalGas', has_market=False),
 Object(id='Steel', has_market=False)]

In [15]:
flows = solution_to_flows(model, {"Z_1": 10, "Z_2": 2, "a_1": 0.5})
flows

,source,target,material,value
0,CCGT,Electricity,Electricity,8.00000000000000
1,HydrogenElectrolysis,Hydrogen,Hydrogen,5.00000000000000
2,SteelProductionEAF,Steel,Steel,5.00000000000000
3,SteelProductionH2DRI,Steel,Steel,5.00000000000000
4,WindTurbine,Electricity,Electricity,2.00000000000000
5,NaturalGas,CCGT,NaturalGas,8.00000000000000
6,Electricity,ElectricCarUse,Electricity,0
7,Electricity,HydrogenElectrolysis,Electricity,5.00000000000000
8,Electricity,SteelProductionEAF,Electricity,5.00000000000000
9,Hydrogen,SteelProductionH2DRI,Hydrogen,5.00000000000000


In [16]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
w = SankeyWidget(links=flows.to_dict(orient='records'), layout=Layout(width="1000", height="300"))
w.order = [
    ["NaturalGas"],
    ["CCGT", "WindTurbine"],
    ["Electricity"],
    ["HydrogenElectrolysis"],
    ["Hydrogen"],
    ["SteelProductionH2DRI", "SteelProductionEAF",],
    ["Steel"],
]
w

SankeyWidget(layout=Layout(height='300', width='1000'), links=[{'source': 'CCGT', 'target': 'Electricity', 'ma…

In [9]:
from ipywidgets import interact

@interact(d1=(0., 10.), d2=(0., 10.), a1=(0., 1.))
def calc_flows(d1=5.0, d2=2.0, a1=0.5):
    flows = solution_to_flows(model, {"Z_1": d1, "Z_2": d2, "a_1": a1})
    w.links = flows.to_dict(orient='records')

interactive(children=(FloatSlider(value=5.0, description='d1', max=10.0), FloatSlider(value=2.0, description='…

In [10]:
from IPython.display import display

In [11]:
for j, sym in model.Y.items():
    print("Process output: %s" % model.processes[j].id)
    display(model[sym])
    print("History:", "\n".join(model.get_history(sym)))
    print("\n")

Process output: CCGT


Max(0, -S_0,4*Piecewise((0, U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3) <= 0), (Z_2, Z_2 <= U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3)), ((U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3))/S_0,4, True)) + U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3))/S_0,0

History: Supply from CCGT (second choice)


Process output: HydrogenElectrolysis


U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3)

History: Demand for steel


Process output: SteelProductionEAF


...

History: Demand for steel


Process output: SteelProductionH2DRI


Z_1*(1 - a_1)/S_3,3

History: Demand for steel


Process output: WindTurbine


Piecewise((0, U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3) <= 0), (Z_2, Z_2 <= U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3)), ((U_0,2*Z_1*a_1/S_3,2 + U_0,1*U_1,3*Z_1*(1 - a_1)/(S_1,1*S_3,3))/S_0,4, True))

History: Supply from wind turbines (first choice)


